# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import json

In [4]:
from pymongo import MongoClient
client = MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [3]:
collection.delete_many({})

In [5]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09
...,...,...,...,...,...,...,...,...,...,...,...
7987,xlu6i6lT_vk,How Do MASSIVE Sinkholes Form?,Life Noggin,27,sinkhole|how do sinkholes form|sinkhole in wat...,440393,14362,390,1575,https://i.ytimg.com/vi/xlu6i6lT_vk/default.jpg,22.10
7988,qRoVlH1OcI4,Trump slams Clinton for defending NFL anthem p...,Business Insider,25,Business Insider|Donald Trump|Hillary Clinton|...,55762,1265,760,1873,https://i.ytimg.com/vi/qRoVlH1OcI4/default.jpg,22.10
7989,EoejGgUNmVU,LP - Lost On You (A Night at The McKittrick Ho...,LP,10,LP|Death Valley|Other People|Lost On You|The M...,142908,7088,68,437,https://i.ytimg.com/vi/EoejGgUNmVU/default.jpg,22.10
7990,MT1CMTI0EVw,Tré Melvin @ #YouTubeBlack FanFest Washington ...,YouTube FanFest,24,YouTube FanFest|#YTFF|Washington DC|USA|YTFF|#...,24532,2148,77,0,https://i.ytimg.com/vi/MT1CMTI0EVw/default.jpg,22.10


## Question 0

### Netoyer les données

In [6]:
dict_ytb = df_youtube.to_dict('records') #chaque ligne du df devient un dict
dict_ytb

[{'video_id': 'XpVt6Z1Gjjo',
  'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
  'channel_title': 'Logan Paul Vlogs',
  'category_id': 24,
  'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
  'views': 4394029,
  'likes': 320053,
  'dislikes': 5931,
  'comment_total': 46245,
  'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
  'date': 13.09},
 {'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views':

In [7]:
with open("./data/US_category_id.json","r") as read_file:
    dict_cat = json.load(read_file)    
dict_cat

{'kind': 'youtube#videoCategoryListResponse',
 'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJvJAAShlR6hM"',
 'items': [{'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKmPBggty2mZQ"',
   'id': '1',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Film & Animation',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"',
   'id': '2',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Autos & Vehicles',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
   'id': '10',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
    'title': 'Music',
    'assignable': True}},
  {'kind': 'youtube#videoCategory',
   'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-oBJavSGkfDI"',
   'id': '15',
   'snippet': {'channelId': 'UCBR8-60-B28hp2BmDPdnt

In [11]:
# on a dict_ytb et dict_cat qui sont des dictionnaires

# on fait une double boucle permettant de remplacer category_id par le title de l'id
for i in dict_ytb :
    for j in dict_cat['items'] :
        if i['category_id'] == int(j['id']) :
            i['category_name'] = j['snippet']['title']

In [12]:
#vérification
dict_ytb[0]

{'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': 'logan paul vlog|logan paul|logan|paul|olympics|logan paul youtube|vlog|daily|comedy|hollywood|parrot|maverick|bird|maverick clothes|diamond play button|logan paul diamond play button|10M subscribers|logan paul 1 year vlogging|1 year vlog|dwarf mamba play button|logan paul history|youtube history|10M|10M plaque|youtube button|diamond button|logang|logang 4 life',
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09,
 'category_name': 'Entertainment'}

In [13]:
# Question 3, mais maintenant :

for i in dict_ytb:
    i['tags'] = i['tags'].split("|")

### Importer les données

In [14]:
collection.insert_many(dict_ytb)

In [15]:
#verification
collection.find_one()

{'_id': ObjectId('5fc51f94c4d2669930414e61'),
 'video_id': 'XpVt6Z1Gjjo',
 'title': '1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED YOUTUBE FOREVER!',
 'channel_title': 'Logan Paul Vlogs',
 'category_id': 24,
 'tags': ['logan paul vlog',
  'logan paul',
  'logan',
  'paul',
  'olympics',
  'logan paul youtube',
  'vlog',
  'daily',
  'comedy',
  'hollywood',
  'parrot',
  'maverick',
  'bird',
  'maverick clothes',
  'diamond play button',
  'logan paul diamond play button',
  '10M subscribers',
  'logan paul 1 year vlogging',
  '1 year vlog',
  'dwarf mamba play button',
  'logan paul history',
  'youtube history',
  '10M',
  '10M plaque',
  'youtube button',
  'diamond button',
  'logang',
  'logang 4 life'],
 'views': 4394029,
 'likes': 320053,
 'dislikes': 5931,
 'comment_total': 46245,
 'thumbnail_link': 'https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg',
 'date': 13.09,
 'category_name': 'Entertainment'}

## Question 1  

In [16]:
#toutes les vidéos de la chaîne Apple
cur = collection.find({"channel_title":"Apple"})
list(cur)

[{'_id': ObjectId('5fc51f94c4d2669930414e62'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
   'Apple Pay',
   'camera',
   'smartphone'],
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09,
  'category_name': 'Science & Technology'},
 {'_id': ObjectId('5fc51f94c4d2669930414f2c'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
   'Apple Pay',
   'camera',
   'smartph

## Question 2

In [17]:
#le nombre de catégories différentes
cat = collection.distinct('category_name')
len(cat)

## ou bien en plus long :
# cur = collection.aggregate([{"$group" : {"_id" : "$category_id", "nombre" : {"$sum" : 1}}}])
# len(list(cur))

16

## Question 3

In [ ]:
# Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents 
# avec une requête update

# Voir la fin de 'Nettoyage des données'


## Question 4

In [18]:
#4 Récupérer les vidéos les plus vues

#TOP 15 des videos les plus vues :
cur = collection.find().sort([("views",-1)]).limit(15)

list(cur)

[{'_id': ObjectId('5fc51f94c4d26699304155d4'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09,
  'category_name': 'Music'},
 {'_id': ObjectId('5fc51f94c4d26699304154ff'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09,
  'category_name': 'Music'},
 {'_id': ObjectId('5fc51f94c4d2669930414f12'),
  'video_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'c

## Question 5

In [19]:
# Compter le nombre moyen de vues en fonction de la catégorie

cur = collection.aggregate([{"$group" : {"_id" : "$category_name", "average views" : {"$avg" : "$views"}}}])
list(cur)


[{'_id': 'Pets & Animals', 'average views': 651404.3879310344},
 {'_id': 'Nonprofits & Activism', 'average views': 1110334.2142857143},
 {'_id': 'Shows', 'average views': 8492.5},
 {'_id': 'Howto & Style', 'average views': 537665.1806674339},
 {'_id': 'Entertainment', 'average views': 1154868.4397251718},
 {'_id': 'Travel & Events', 'average views': 464041.0833333333},
 {'_id': 'Science & Technology', 'average views': 924730.548828125},
 {'_id': 'Film & Animation', 'average views': 1039472.6560846561},
 {'_id': 'Music', 'average views': 1176553.612},
 {'_id': 'Education', 'average views': 547582.4700598803},
 {'_id': 'People & Blogs', 'average views': 971532.7573696146},
 {'_id': 'Sports', 'average views': 728434.3243902439},
 {'_id': 'Autos & Vehicles', 'average views': 607693.8706896552},
 {'_id': 'News & Politics', 'average views': 540955.5569823436},
 {'_id': 'Gaming', 'average views': 681081.8170731707},
 {'_id': 'Comedy', 'average views': 1240073.294039735}]

## Question 6 

In [26]:
# Récupérer les chaines Youtube avec la plus grande moyenne de likes

# TOP 15 des chaînes youtube avec le plus grand nombre de likes 

cur = collection.aggregate([
         {"$group" : {"_id" : "$channel_title", "Average likes" : {"$avg" :"$likes"}}},
         { '$sort': { "Average likes": -1 }},
         { '$limit' : 15 }
     ])

list(cur)

[{'_id': 'ZaynVEVO', 'Average likes': 1431683.0},
 {'_id': 'ibighit', 'Average likes': 1371766.25},
 {'_id': 'melanie martinez', 'Average likes': 911871.5},
 {'_id': 'BETNetworks', 'Average likes': 769687.625},
 {'_id': 'jypentertainment', 'Average likes': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'Average likes': 653443.5384615385},
 {'_id': 'Wengie', 'Average likes': 567235.2},
 {'_id': 'Alesso', 'Average likes': 509417.28571428574},
 {'_id': 'PewDiePie', 'Average likes': 508647.8},
 {'_id': 'shakiraVEVO', 'Average likes': 434302.14285714284},
 {'_id': 'Ed Sheeran', 'Average likes': 395654.1666666667},
 {'_id': 'Dude Perfect', 'Average likes': 389697.1111111111},
 {'_id': 'Conor Maynard', 'Average likes': 389273.0},
 {'_id': 'TaylorSwiftVEVO', 'Average likes': 367455.6666666667},
 {'_id': 'Liza Koshy', 'Average likes': 365146.9583333333}]